# Ottimizzazione iperparametri TPE
Hyperopt include un metodo di ricerca più sofisticato delle semplici grid search e random search, chiamato Tree-structured Parzen Estimator (TPE), che cerca dinamicamente di ridurre lo spazio dove ricercare le soluzioni ottimali per massimizzare il numero di chiamate alla funzione.

Importazione di tutte le librerie.

In [7]:
import scipy
import IPython
import sklearn
import tensorflow
import keras
import pylab as pl
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from matplotlib.lines import Line2D
from scipy.interpolate import interp1d
import scipy.odr as odr 
from keras.activations import relu
from keras.activations import tanh
import matplotlib as mp
import itertools
import random
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from array import array
from sklearn import metrics
from keras.utils import to_categorical
from sklearn.utils import class_weight
from sklearn import svm, datasets
from scipy import interp
from keras.callbacks import History 
history = History()
from keras.callbacks import *
from sklearn.externals import joblib
from scipy.stats import norm
from sklearn.model_selection import GridSearchCV
import scipy.stats
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.metrics import mean_absolute_error
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.layers.advanced_activations import LeakyReLU, PReLU
from hyperopt import fmin, tpe, hp, Trials
from hyperopt import STATUS_OK
from keras.layers.normalization import BatchNormalization

Importiamo i dati su cui performare la regressione e dividiamo il dataset in training set validation set e test set:

In [8]:
ewk_input = np.load('ewk_signal_withreco_withdelta_input.npy')
ewk_truth = np.load('ewk_signal_withreco_withdelta_truth.npy')
ewk_input_train = ewk_input[0:400000,:]
ewk_truth_train = ewk_truth[0:400000,4].reshape(-1, 1)

ewk_input_validation = ewk_input[400000:450000,:]
ewk_truth_validation = ewk_truth[400000:450000,4].reshape(-1, 1)

ewk_input_test = ewk_input[450000:501328,:]
ewk_truth_test = ewk_truth[450000:501328,4].reshape(-1, 1)

Z-scaliamo il dataset in modo da ottenere dati più maneggiabili poichè sono più vicini allo 0.
Z-scalare vuol dire fare la seguente cosa:
$ data_{new} = \frac{data_{old}-\mu}{\sigma} $
Salviamo successivamente il modello scalato in un file 'scaler_input.pkl' in modo da poterlo richiamare successivamente

In [9]:
scaler_input = StandardScaler()
scaler_truth = StandardScaler()


scaler_input.fit(ewk_input_train)
scaler_truth.fit(ewk_truth_train)

input_train = scaler_input.transform(ewk_input_train)
truth_train = scaler_truth.transform(ewk_truth_train)
input_validation = scaler_input.transform(ewk_input_validation)
truth_validation = scaler_truth.transform(ewk_truth_validation)
input_test = scaler_input.transform(ewk_input_test)
truth_test = ewk_truth_test

joblib.dump(scaler_input,  "scaler_input.pkl")
joblib.dump(scaler_truth,  "scaler_truth.pkl")

['scaler_truth.pkl']

Definiamo adesso la cartella in cui salvare i file:

In [10]:
output_dir = "/home/christian/Scrivania/notebook/VBS analysis/risultati_predizioni/grid_search"
comparison_dir = "/home/christian/Scrivania/notebook/VBS analysis/risultati_predizioni"

Per comodità definiamo due funzioni: la prima, train_fcn per allenare il modello (una rete neurale con uno strato nascosto), e la seconda, test_fcn, per valutarne le prestazioni. Cominciamo dalla prima:

In [51]:
def train_fcn(features, labels, train_params):

  model = Sequential()
  model.add(Dense(units=int(train_params['layer_size']), activation='relu', input_dim=input_train.shape[1]))
  model.add(Dense(units=1, activation='softmax'))

  adam = Adam(lr=train_params['learning_rate'])
  model.compile(loss='mse', optimizer=adam, metrics=['mse'])
  model.fit(features, labels, epochs=50, batch_size=32, verbose=0)

  return model

def test_fcn(model, features, labels):
  test_accuracy = model.evaluate(features, labels)
  return test_accuracy

Come prima cosa, definiamo uno spazio di ricerca per gli iperparametri. Oltre a hp.uniform, abbiamo una vasta scelta di configurazioni possibili, a seconda dell'iperparametro. Per ora ne introduciamo altri due:

    hp.choice per selezionare layer_size in un intervallo discreto di scelte, ovvero nell'intervallo [5,10,⋯,25]


    hp.loguniform per cercare il learning rate in un intervallo esponenziale exp(j) con j∈[−10,0]

Questo perché iperparametri come il learning rate sono spesso poco sensibili a piccole variazioni e richiedono di valutare numerosi ordini di grandezza simultaneamente.

Unendo tutto quanto all'interno di un dizionario:

In [52]:
search_space = {
  'layer_size': hp.choice('layer_size', np.arange(5, 26, 5)),
  'learning_rate': hp.loguniform('learning_rate', -10, 0),
}

La funzione da ottimizzare è molto semplice: per ogni configurazione di iperparametri da provare allena un modello sulla parte rimanente di training, e ne valuta le prestazioni su quella di validazione:

In [53]:
from keras import backend as K
def hyperopt_fcn(params):
  model = train_fcn(input_train, truth_train, params)
  test_acc = test_fcn(model, input_validation, truth_validation)
  K.clear_session()
  return {'loss': -test_acc[1], 'status': STATUS_OK}

I parametri da testare vengono passati da Hyperopt in forma di dizionario (la stessa convenzione che abbiamo rispettato per la funzione di allenamento).
K.clear_session() evita che i modelli di Keras si accumulino in memoria.
Passiamo ad Hyperopt due informazioni distinte: il valore da minimizzare (l'opposto dell'accuratezza), ed uno "status" con valore OK. Questo valore è molto utile perché permette di distinguere configurazioni che danno vita ad errori e/o problemi numerici (come un learning rate molto alto).

In [ ]:
best = fmin(hyperopt_fcn, search_space, algo=tpe.suggest, max_evals=50)

   32/50000 [..............................]        
 - ETA: 1:30                                        
                                                   
 1056/50000 [..............................]        
 - ETA: 5s                                          
                                                   
 2016/50000 [>.............................]        
 - ETA: 3s                                          
                                                   
 3328/50000 [>.............................]        
 - ETA: 2s                                          
                                                   
 4128/50000 [=>............................]        
 - ETA: 2s                                          
                                                   
 5152/50000 [==>...........................]        
 - ETA: 2s                                          
                                                   
 6176/50000 [==>...........................]        

                                                   
50000/50000 [==============================]        
 - 3s 54us/step                                     

   32/50000 [..............................]                                      
 - ETA: 1:06                                                                      
                                                                                 
 1088/50000 [..............................]                                      
 - ETA: 4s                                                                        
                                                                                 
 2304/50000 [>.............................]                                      
 - ETA: 3s                                                                        
                                                                                 
 3712/50000 [=>............................]                                      
 - ETA: 2s    

 - ETA: 0s                                                                        
                                                                                 
39936/50000 [======================>.......]                                      
 - ETA: 0s                                                                        
                                                                                 
41120/50000 [=======================>......]                                      
 - ETA: 0s                                                                        
                                                                                 
42368/50000 [========================>.....]                                      
 - ETA: 0s                                                                        
                                                                                 
44000/50000 [=========================>....]                                      
 - ETA: 

                                                                                 
15232/50000 [========>.....................]                                      
 - ETA: 2s                                                                        
                                                                                 
16032/50000 [========>.....................]                                      
 - ETA: 2s                                                                        
                                                                                 
16864/50000 [=========>....................]                                      
 - ETA: 2s                                                                        
                                                                                 
17760/50000 [=========>....................]                                      
 - ETA: 2s                                                                        
        

                                                                                 
40224/50000 [=======================>......]                                      
 - ETA: 0s                                                                        
                                                                                 
41152/50000 [=======================>......]                                      
 - ETA: 0s                                                                        
                                                                                 
42176/50000 [========================>.....]                                      
 - ETA: 0s                                                                        
                                                                                 
43264/50000 [========================>.....]                                      
 - ETA: 0s                                                                        
        

 - ETA: 3s                                                                          
                                                                                   
12640/50000 [======>.......................]                                        
 - ETA: 3s                                                                          
                                                                                   
13568/50000 [=======>......................]                                        
 - ETA: 3s                                                                          
                                                                                   
14784/50000 [=======>......................]                                        
 - ETA: 3s                                                                          
                                                                                   
15648/50000 [========>.....................]                         

 - ETA: 0s                                                                          
                                                                                   
41120/50000 [=======================>......]                                        
 - ETA: 0s                                                                          
                                                                                   
42144/50000 [========================>.....]                                        
 - ETA: 0s                                                                          
                                                                                   
42976/50000 [========================>.....]                                        
 - ETA: 0s                                                                          
                                                                                   
44448/50000 [=========================>....]                         

                                                                                   
16512/50000 [========>.....................]                                        
 - ETA: 2s                                                                          
                                                                                   
17280/50000 [=========>....................]                                        
 - ETA: 2s                                                                          
                                                                                   
17952/50000 [=========>....................]                                        
 - ETA: 2s                                                                          
                                                                                   
18784/50000 [==========>...................]                                        
 - ETA: 2s                                                           

                                                                                   
45792/50000 [==========================>...]                                        
 - ETA: 0s                                                                          
                                                                                   
46560/50000 [==========================>...]                                        
 - ETA: 0s                                                                          
                                                                                   
47776/50000 [===========================>..]                                        
 - ETA: 0s                                                                          
                                                                                   
49280/50000 [============================>.]                                        
 - ETA: 0s                                                           

24352/50000 [=============>................]                                        
 - ETA: 1s                                                                          
                                                                                   
25120/50000 [==============>...............]                                        
 - ETA: 1s                                                                          
                                                                                   
25920/50000 [==============>...............]                                        
 - ETA: 1s                                                                          
                                                                                   
27200/50000 [===============>..............]                                        
 - ETA: 1s                                                                          
                                                                    

Possiamo valutare la configurazione ottimale ritornata dall'algoritmo:

In [ ]:
from hyperopt import space_eval
space_eval(search_space, best)

Possiamo quindi procedere ad una fase finale di training, usando gli iperparametri ottimali ma riunendo la componente di validazione a quella di training:

In [ ]:
model = train_fcn(np.vstack((input_train, input_validation)), np.vstack((truth_train, truth_val)), space_eval(search_space, best))
print('MSE is: ', test_fcn(model, input_test, truth_test))

Hyperopt permette di parallelizzare il processo di ricerca sfruttando MongoDB per salvare i risultati intermedi. 

In [ ]:
trials = Trials()
best = fmin(hyperopt_fcn, search_space, algo=tpe.suggest, max_evals=50, trials=trials)
print(trials.trials[0])

Da questo oggetto possiamo ottenere moltissime informazioni utili. Prima di tutto, possiamo stampare il valore di loss function ottenuta da ciascun esperimento:

In [ ]:
plt.figure()
xs = [t['tid'] for t in trials.trials]
ys = [-t['result']['loss'] for t in trials.trials]
plt.xlim(xs[0]-1, xs[-1]+1)
plt.scatter(xs, ys, s=20, linewidth=0.01, alpha=0.75)
plt.xlabel('Iteration', fontsize=16)
plt.ylabel('MSE', fontsize=16)
plt.show()